In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import HDBSCAN
from sklearn.datasets import load_digits
from sklearn.metrics import v_measure_score
import pickle
import jsonrpclib
import random
import time
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from keras.models import load_model
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, mean_absolute_percentage_error, r2_score

2023-08-22 08:29:19.214911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 08:29:19.279262: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 08:29:19.280276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 08:29:20.008541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Przyjęte parametry regulatora pid 
Kc   = 3.73 # człon proporcjonalny 
tauI = 72 # czas zdwojenie
tauD = 12   # człon wyprzedzenia


def pid(sp,pv,pv_last,ierr,dt):
    KP = Kc
    KI = Kc / tauI
    KD = Kc * tauD
    
    op0 = 0 # Wartość początkowa sterowania
    
    # Górna i dolna granica mocy grzałki
    ophi = 100
    oplo = 0
    
    error = sp - pv # Obliczenia błedu
    
    # Obliczenie całki błędu
    ierr = ierr + KI * error * dt
    
    # Obliczenie pochodnej pomiaru
    if dt>=1e-8:
        dpv = (pv - pv_last) / dt
    else:
        dpv = 0.0;
    
    # Obliczenie wartości PID
    P = KP * error
    #I=0
    I = ierr
    D = -KD * dpv
    op = op0 + P + I + D
    
    # Ograniczenie wartości sterowania
    if op < oplo or op > ophi:
        I = I - KI * error * dt
        op = max(oplo,min(ophi,op))
    op = round(op, 0)
    #I=0   
    # Zwrócenie wartości sterowania i składowych PID
    return [op,P,I,D]

In [ ]:
s=jsonrpclib.Server('http://10.20.1.75:8080/rpc') # COnnetc to Object

In [ ]:
# Parametry symulacji
f = 5
# run_time = 9*60.0 #[min]
run_time = 3.5*60.0 #[min]

loops = int((60.0/f)*run_time)

print("Run time:", run_time, "loops:", loops)

# Tablice do przechowywania danych
T1 = np.zeros(loops)    # Mierzona temperatura [C]
Q1 = np.zeros(loops)    # Wartość mocy grzewczej
tm = np.zeros(loops)    # Czas
pomp = np.zeros(loops)  #Załączanie pomp
# Tablica z watościami nastawionej temperatury
Tsp1 = np.ones(loops) *40
      
#Zmiana wartośći nastawionej temperatury
t2 = 0
while t2 <= loops:
    t1 = t2
    t2 += random.randint(400,500) 
    Tsp1[t1:t2] = 40 + random.randint(0,20)
    #Tsp1[t1:t2] = 40
    
    
fig = make_subplots(rows=1, cols = 1)


fig.add_trace(
    go.Scatter(y=Tsp1, name="Temperatura nastawiona"), row=1, col=1
)
fig.show()


In [ ]:
ierr = 0
zal_pomp = 0    
end_time = 0
for i in range(loops): 
    start_time = time.time()  # pobieramy czas rozpoczęcia iteracji
    tm[i] = start_time          
    # Odczyt temperatury
    a = s.ai_get('1_01')
    T1[i] = 18*a['value']-30

    # Oblicz wyjście regulatora PID
    dt = ( start_time - end_time )
    [Q1[i],P,ierr,D] = pid(Tsp1[i],T1[i],T1[i-1],ierr,5) #"""Tsp1[i]"""
    x = Q1[i]/100
    q = 157.35*x*x*x*x*x - 403.71*x*x*x*x + 384.79*x*x*x - 166.42*x*x + 40.785*x + 5.1684
    print(i, "T_Set:",Tsp1[i],"T:", T1[i],"q[mA]:", q,"q[%]:", (x*100), "dt", dt, end_time, start_time)
    s.ao_set_value("1_01", q)
           
    end_time = time.time()  # pobieramy czas zakończenia iteracji
    elapsed_time = end_time - start_time  # obliczamy czas trwania iteracji
    if elapsed_time < 5:  # jeśli iteracja trwała krócej niż 5 sekund
        time.sleep(5 - elapsed_time)  # wstrzymujemy wykonywanie programu na pozostały czas
s.ao_set_value("1_01", 0)

In [ ]:
fig = make_subplots(rows=2, cols = 1)

fig.add_trace(
    go.Scatter(y=Tsp1[:i], name="Temperatura zmierzona"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y= T1[:i], name="Temperatura nastawiona"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y=Q1[:i], name="Sterowanie"), row=2, col=1
)


fig['layout']['yaxis']['title']=r'Temperatura [$^oC$]'
fig['layout']['yaxis2']['title']=r'Wartość mocy grzewczej [%]'
fig['layout']['xaxis2']['title']=r'Czas [s]'   

fig.update_layout(height=600, width=800, title_text="Sterowanie rzeczywistym układem za pomocą kontrolera PID")
fig.show()

In [ ]:
df = pd.DataFrame({'temperatura': T1, 'regulacja':Q1, 'nastaw_t':Tsp1})
df.to_csv('PID_garnek_6th_try.csv', index=False)

In [ ]:
### SCENARIUSZ TESTOWY

In [ ]:

import pickle
pickled_model = pickle.load(open('SVM.pkl', 'rb'))

In [ ]:
# Parametry symulacji
df = pd.read_csv('Data/PID_garnek_6th_try.csv')
Tsp1 = df['nastaw_t'].to_numpy()
loops = len(Tsp1)
f = 5
run_time = len(Tsp1)/3600*60.0*5 #[min]


print("Run time:", run_time, "loops:", loops)

# Tablice do przechowywania danych
T1 = np.zeros(loops)    # Mierzona temperatura [C]
Q1 = np.zeros(loops)
Q1_pid = np.zeros(loops) # Wartość mocy grzewczej
tm = np.zeros(loops)    # Czas

fig = make_subplots(rows=1, cols = 1)

fig.add_trace(
    go.Scatter(y=Tsp1, name="Temperatura nastawiona"), row=1, col=1
)
fig.show()


In [ ]:
ierr = 0
zal_pomp = 0    
end_time = 0
for i in range(loops): 
    start_time = time.time()  # pobieramy czas rozpoczęcia iteracji
    tm[i] = start_time          
    # Odczyt temperatury
    a = s.ai_get('1_01')
    T1[i] = 18*a['value']-30

    # Oblicz wyjście regulatora PID
    dt = ( start_time - end_time )
    error = Tsp1[i] - T1[i]
    # Q1[i] = model(error)
    Q1[i] = pickled_model.predict([[error]])
    [Q1_pid[i],P,ierr,D] = pid(Tsp1[i],T1[i],T1[i-1],ierr,5)
    x = Q1[i]/100
    q = 157.35*x*x*x*x*x - 403.71*x*x*x*x + 384.79*x*x*x - 166.42*x*x + 40.785*x + 5.1684
    if q>100:
        q = 100
    elif q<0:
        q = 0
    print(i, "T_Set:",Tsp1[i],"T:", T1[i],"q[mA]:", q,"q[%]:", (x*100), "dt", dt, end_time, start_time)
    s.ao_set_value("1_01", q)
           
    end_time = time.time()  # pobieramy czas zakończenia iteracji
    elapsed_time = end_time - start_time  # obliczamy czas trwania iteracji
    if elapsed_time < 5:  # jeśli iteracja trwała krócej niż 5 sekund
        time.sleep(5 - elapsed_time)  # wstrzymujemy wykonywanie programu na pozostały czas
s.ao_set_value("1_01", 0)

In [ ]:
fig = make_subplots(rows=2, cols = 1)

fig.add_trace(
    go.Scatter(y=Tsp1[:i], name="Temperatura zmierzona"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y= T1[:i], name="Temperatura nastawiona"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y=Q1[:i], name="Sterowanie"), row=2, col=1
)


fig['layout']['yaxis']['title']=r'Temperatura [$^oC$]'
fig['layout']['yaxis2']['title']=r'Wartość mocy grzewczej [%]'
fig['layout']['xaxis2']['title']=r'Czas [s]'   

fig.update_layout(height=600, width=800, title_text="Sterowanie rzeczywistym układem za pomocą kontrolera PID")
fig.show()

In [ ]:
df = pd.DataFrame({'temperatura': T1, 'regulacja':Q1, 'regulacja_pid':Q1_pid, 'nastaw_t':Tsp1})
df.to_csv('Data/SVM_garnek_2th_try.csv', index=False)

In [ ]:
## ZBIERANIE DANYCH DIGITAL TWIN

In [ ]:
# Parametry symulacji
f = 5
# run_time = 9*60.0 #[min]
run_time = 1.5*60.0 #[min]

loops = int((60.0/f)*run_time)

print("Run time:", run_time, "loops:", loops)

# Tablice do przechowywania danych
T1 = np.zeros(loops)    # Mierzona temperatura [C]
Q1 = np.zeros(loops)    # Wartość mocy grzewczej
tm = np.zeros(loops)    # Czas
  
#Zmiana wartośći nastawionej temperatury
t2 = 0

while t2 <= loops:
    t1 = t2
    t2 += random.randint(80,150) 
    # Q1[t1:t2] = random.randint(0,100)
    Q1[t1:t2] = random.randint(40,40)
  
fig = make_subplots(rows=1, cols = 1)


fig.add_trace(
    go.Scatter(y=Q1, name="Regulacja %"), row=1, col=1
)
fig.show()

In [ ]:
ierr = 0  
end_time = 0
for i in range(loops): 
    start_time = time.time()  # pobieramy czas rozpoczęcia iteracji
    tm[i] = start_time          
    # Odczyt temperatury
    a = s.ai_get('1_01')
    T1[i] = 18*a['value']-30

    # Oblicz wyjście regulatora PID
    dt = ( start_time - end_time )


    x = Q1[i]/100
    q = 157.35*x*x*x*x*x - 403.71*x*x*x*x + 384.79*x*x*x - 166.42*x*x + 40.785*x + 5.1684

    print(i,"T:", T1[i],"q[mA]:", q,"q[%]:", (x*100), "dt", dt, end_time, start_time)
    s.ao_set_value("1_01", q)
           
    end_time = time.time()  # pobieramy czas zakończenia iteracji
    elapsed_time = end_time - start_time  # obliczamy czas trwania iteracji
    if elapsed_time < 5:  # jeśli iteracja trwała krócej niż 5 sekund
        time.sleep(5 - elapsed_time)  # wstrzymujemy wykonywanie programu na pozostały czas
s.ao_set_value("1_01", 0)

In [ ]:
fig = make_subplots(rows=2, cols = 1)

fig.add_trace(
    go.Scatter(y=Tsp1[:i], name="Temperatura zmierzona"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y=Q1[:i], name="Sterowanie"), row=2, col=1
)

fig['layout']['yaxis']['title']=r'Temperatura [$^oC$]'
fig['layout']['xaxis2']['title']=r'Czas [s]'   

fig.update_layout(height=600, width=800, title_text="Sterowanie rzeczywistym układem za pomocą kontrolera PID")
fig.show()

In [ ]:
df = pd.DataFrame({'temperature': T1, 'control': Q1})
df.to_csv('Data/DT_garnek_1th_try_continue_3', index=False)

In [ ]:
s.ao_set_value("1_01", 9.151336)

In [ ]:
s.ao_set_value("1_01", 0)

In [ ]:
# SYMULATOR

In [4]:
# Parametry symulacji
# df = pd.read_csv('PID_garnek_6th_try.csv')
# Tsp1 = df['nastaw_t'].to_numpy()
# loops = len(Tsp1)
# f = 5
# run_time = len(Tsp1)/3600*60.0*5 #[min]


# # Parametry symulacji
f = 5
# run_time = 9*60.0 #[min]
run_time = 5*60.0 #[min]
loops = int((60.0/f)*run_time)
model = load_model('Models/LSTM_DT.keras')
window = 30
s1 = pickle.load(open('Models/input_scaler.sav', 'rb'))
s2 = pickle.load(open('Models/output_scaler.sav', 'rb'))

print("Run time:", run_time, "loops:", loops)

# Tablice do przechowywania danych
T = np.concatenate((np.ones(window)*25, np.zeros(loops - window) )) # Mierzona temperatura [C]
T1 = np.concatenate((np.ones(window)*25, np.zeros(loops - window) )) # Mierzona temperatura [C]
Q = np.zeros(loops) 
Q1 = np.zeros(loops)    # Wartość mocy grzewczej

# Tablica z watościami nastawionej temperatury
Tsp = np.ones(loops)
      
#Zmiana wartośći nastawionej temperatury
t2 = 0
while t2 <= loops:
    t1 = t2
    t2 += random.randint(150,250) 
    Tsp[t1:t2] = 40 + random.randint(0,20)
    
    
fig = make_subplots(rows=1, cols = 1)


fig.add_trace(
    go.Scatter(y=Tsp, name="Temperatura nastawiona"), row=1, col=1
)
fig.show()


Run time: 300.0 loops: 3600


In [5]:
Xt = []
Yt = []
for i in range (loops):
    Xt.append([T[i], Q[i]])

Xt = np.array(Xt)
Xts = s1.transform(Xt)

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



In [6]:
# PID

ierr = 0
zal_pomp = 0    
end_time = 0
for i in range(window, loops): 
    Xin = Xts[i-window:i].reshape((1, window, 2))
    test = []
    Xts[i][0] = model.predict(Xin)
    temp = s1.inverse_transform([Xts[i]])
    # print(temp[0][0])
    T[i] = temp[0][0]
    [Q[i],P,ierr,D] = pid(Tsp[i],T[i],T[i-1],ierr,5)
    test.append([T[i], Q[i]])
    test = np.array(test)
    Xts[i] = s1.transform(test)

    print(i, "T_Set:",Tsp[i],"T:", T[i],"q[%]:", Q[i])

1/1 [==============================] - 1s 770ms/step
30 T_Set: 45.0 T: 26.232187290563928 q[%]: 64.0
1/1 [==============================] - 0s 21ms/step
31 T_Set: 45.0 T: 26.23946366567514 q[%]: 80.0
1/1 [==============================] - 0s 20ms/step
32 T_Set: 45.0 T: 26.254232614902463 q[%]: 84.0
1/1 [==============================] - 0s 19ms/step
33 T_Set: 45.0 T: 26.27326277677479 q[%]: 89.0
1/1 [==============================] - 0s 17ms/step
34 T_Set: 45.0 T: 26.299139240469003 q[%]: 94.0


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 [==============================] - 0s 18ms/step
35 T_Set: 45.0 T: 26.34043557512573 q[%]: 98.0
1/1 [==============================] - 0s 17ms/step
36 T_Set: 45.0 T: 26.408759110115664 q[%]: 100.0
1/1 [==============================] - 0s 16ms/step
37 T_Set: 45.0 T: 26.515582341021503 q[%]: 100.0
1/1 [==============================] - 0s 17ms/step
38 T_Set: 45.0 T: 26.670255293625118 q[%]: 100.0
1/1 [==============================] - 0s 16ms/step
39 T_Set: 45.0 T: 26.8804913904885 q[%]: 100.0
1/1 [==============================] - ETA: 0s

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 [==============================] - 0s 17ms/step
40 T_Set: 45.0 T: 27.151979222633095 q[%]: 100.0
1/1 [==============================] - 0s 17ms/step
41 T_Set: 45.0 T: 27.488194247180708 q[%]: 100.0
1/1 [==============================] - 0s 17ms/step
42 T_Set: 45.0 T: 27.890401112073995 q[%]: 98.0
1/1 [==============================] - 0s 16ms/step
43 T_Set: 45.0 T: 28.356161185526478 q[%]: 100.0
1/1 [==============================] - 0s 16ms/step
44 T_Set: 45.0 T: 28.884718933381247 q[%]: 98.0
1/1 [==============================] - ETA: 0s

...

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 [==============================] - 0s 16ms/step
3589 T_Set: 41.0 T: 48.90741324168195 q[%]: 0.0
1/1 [==============================] - 0s 15ms/step
3590 T_Set: 41.0 T: 48.90176649562919 q[%]: 0.0
1/1 [==============================] - 0s 16ms/step
3591 T_Set: 41.0 T: 48.89614067206078 q[%]: 0.0
1/1 [==============================] - 0s 15ms/step
3592 T_Set: 41.0 T: 48.89052879681526 q[%]: 0.0
1/1 [==============================] - 0s 15ms/step
3593 T_Set: 41.0 T: 48.88493784405409 q[%]: 0.0
1/1 [==============================] - 0s 15ms/step
3594 T_Set: 41.0 T: 48.87937013849775 q[%]: 0.0


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 [==============================] - 0s 87ms/step
3595 T_Set: 41.0 T: 48.873816381264305 q[%]: 0.0
1/1 [==============================] - 0s 16ms/step
3596 T_Set: 41.0 T: 48.868281221794724 q[%]: 0.0
1/1 [==============================] - 0s 16ms/step
3597 T_Set: 41.0 T: 48.862769309529966 q[%]: 0.0
1/1 [==============================] - 0s 16ms/step
3598 T_Set: 41.0 T: 48.857269020867626 q[%]: 0.0
1/1 [==============================] - 0s 16ms/step
3599 T_Set: 41.0 T: 48.85178732996915 q[%]: 0.0


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



In [7]:
fig = make_subplots(rows=2, cols = 1)


fig.add_trace(
    go.Scatter(y=T, name="Temperature (LSTM)"), row=1, col=1
)
fig.add_trace(
    go.Scatter(y=Tsp, name="Setpoint"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y=Q, name="PID Control (%)"), row=2, col=1
)
fig.update_layout(height=700, width=1100)
fig.show()

In [13]:
# Ytm_m = []
# Ytu_m = []

# for i in range(len(Ytu)):
#     Ytm_m.append(Ytm[i][0])
#     Ytu_m.append(Ytu[i][0])
    
a = mean_absolute_error(Tsp, T)
b = mean_squared_error(Tsp, T,  squared=True)
c = mean_squared_error(Tsp, T,  squared=False)
d = median_absolute_error(Tsp, T)
e = mean_absolute_percentage_error(Tsp, T)
f = r2_score(Tsp, T)

print("mean_absolute_error = ", a, 
      "mean_squared_error = ", b, 
      "root_mean_squared_error = ", c, 
      "median_absolute_error = ", d,
      "mean_absolute_percentage_error = ", e, 
      "r2_score = ", f)

mean_absolute_error =  5.772128249391238 mean_squared_error =  49.90407461824578 root_mean_squared_error =  7.0642816066636085 median_absolute_error =  5.297294031521584 mean_absolute_percentage_error =  0.12860135448323126 r2_score =  -0.2956687881056643


In [18]:
# SVM
Xt = []
Yt = []
for i in range (loops):
    Xt.append([T1[i], Q1[i]])

Xt = np.array(Xt)
Xts = s1.transform(Xt)
pickled_model = pickle.load(open('Models/SVM.pkl', 'rb'))

ierr = 0
zal_pomp = 0    
end_time = 0
for i in range(window, loops): 
    Xin = Xts[i-window:i].reshape((1, window, 2))
    test = []
    Xts[i][0] = model.predict(Xin)
    temp = s1.inverse_transform([Xts[i]])
    # print(temp[0][0])
    T1[i] = temp[0][0]
    error = Tsp[i] - T1[i]
    Q1[i] = pickled_model.predict([[error]])
    if Q1[i]>100:
        Q1[i] = 100
    elif Q1[i]<0:
        Q1[i] = 0
    test.append([T1[i], Q1[i]])
    test = np.array(test)
    Xts[i] = s1.transform(test)

    print(i, "T_Set:",Tsp[i],"T:", T1[i],"q[%]:", Q1[i])


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 [==============================] - 0s 17ms/step
30 T_Set: 54.0 T: 26.232187290563928 q[%]: 99.93341042484771
1/1 [==============================] - 0s 16ms/step
31 T_Set: 54.0 T: 26.245249894956867 q[%]: 99.92865156947985
1/1 [==============================] - 0s 18ms/step
32 T_Set: 54.0 T: 26.270775325858175 q[%]: 99.9193747230233
1/1 [==============================] - 0s 16ms/step


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



33 T_Set: 54.0 T: 26.304962665278456 q[%]: 99.90746468101473
1/1 [==============================] - 0s 17ms/step
34 T_Set: 54.0 T: 26.351064197173162 q[%]: 99.89370913196456


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



...

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



3592 T_Set: 47.0 T: 50.340340725922886 q[%]: 0.0989485956322369
1/1 [==============================] - 0s 16ms/step
3593 T_Set: 47.0 T: 50.32981671629718 q[%]: 0.09899274834877758
1/1 [==============================] - 0s 17ms/step
3594 T_Set: 47.0 T: 50.319327577478724 q[%]: 0.0990046415672694
1/1 [==============================] - 0s 17ms/step
3595 T_Set: 47.0 T: 50.308875634187984 q[%]: 0.09898210478154823
1/1 [==============================] - 0s 19ms/step
3596 T_Set: 47.0 T: 50.29846088642498 q[%]: 0.09892345560367488


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 [==============================] - 0s 18ms/step
3597 T_Set: 47.0 T: 50.28807868474872 q[%]: 0.09882745483525213
1/1 [==============================] - 0s 17ms/step
3598 T_Set: 47.0 T: 50.27773135387971 q[%]: 0.09869341352943906
1/1 [==============================] - 0s 17ms/step
3599 T_Set: 47.0 T: 50.26742121853843 q[%]: 0.09852120351626326


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



In [21]:
fig = make_subplots(rows=2, cols = 1)

fig.add_trace(
    go.Scatter(y=T1, name="Temperature (LSTM)"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y=Tsp, name="Setpoint"), row=1, col=1
)

fig.add_trace(
    go.Scatter(y=Q1, name="SVM Control (%)"), row=2, col=1
)
fig.update_layout(height=700, width=1100)
fig.show()

In [20]:
a = mean_absolute_error(Tsp, T1)
b = mean_squared_error(Tsp, T1,  squared=True)
c = mean_squared_error(Tsp, T1,  squared=False)
d = median_absolute_error(Tsp, T1)
e = mean_absolute_percentage_error(Tsp, T1)
f = r2_score(Tsp, T1)

print("mean_absolute_error = ", a, 
      "mean_squared_error = ", b, 
      "root_mean_squared_error = ", c, 
      "median_absolute_error = ", d,
      "mean_absolute_percentage_error = ", e, 
      "r2_score = ", f)

mean_absolute_error =  5.882481299834866 mean_squared_error =  56.911774353180064 root_mean_squared_error =  7.543989286390859 median_absolute_error =  6.345066039915331 mean_absolute_percentage_error =  0.13311745396362293 r2_score =  -0.27076729940667543
